In [14]:
import json
import pytrec_eval

In [11]:

results = {
    'q1': {
        'doc1': {'rank': 1, 'score': 0.5},
        'doc2': {'rank': 2, 'score': 0.4},
        'doc3': {'rank': 3, 'score': 0.3},
        'doc4': {'rank': 4, 'score': 0.2},
        'doc5': {'rank': 5, 'score': 0.1},
    },
    'q2': {
        'doc6': {'rank': 1, 'score': 0.6},
        'doc7': {'rank': 2, 'score': 0.5},
        'doc8': {'rank': 3, 'score': 0.4},
        'doc9': {'rank': 4, 'score': 0.3},
        'doc10': {'rank': 5, 'score': 0.2},
    },
    # ...
}

# Create a dictionary of query ids and their corresponding relevance judgements
qrels = {
    'q1': {
        'doc1': 1,
        'doc2': 0,
        'doc3': 1,
        'doc4': 0,
        'doc5': 1,
    },
    'q2': {
        'doc6': 0,
        'doc7': 1,
        'doc8': 1,
        'doc9': 0,
        'doc10': 0,
    },
    # ...
}

# Create a dictionary of query ids and their corresponding information need descriptions
queries = {
    'q1': 'What is the capital of France?',
    'q2': 'Who is the author of The Catcher in the Rye?',
    # ...
}

# Create a dictionary of document ids and their corresponding text contents
documents = {
    'doc1': 'Paris is the capital of France.',
    'doc2': 'France is a country in Europe.',
    'doc3': 'The Eiffel Tower is located in Paris.',
    'doc4': 'Germany is a country in Europe.',
    'doc5': 'The Louvre is a museum located in Paris.',
    'doc6': 'The Catcher in the Rye is a novel by J.D. Salinger.',
    'doc7': 'J.D. Salinger was an American writer.',
    'doc8': 'The Catcher in the Rye is a coming-of-age novel.',
    'doc9': 'To Kill a Mockingbird is a novel by Harper Lee.',
    'doc10': 'Harper Lee was an American novelist.',
    # ...
}

# Create a dictionary of document ids and their corresponding relevance levels for each query
relevance_levels = {
    'q1': {
        'doc1': 1,
        'doc2': 0,
        'doc3': 2,
        'doc4': 0,
        'doc5': 1,
    },
    'q2': {
        'doc6': 2,
        'doc7': 2,
        'doc8': 1,
        'doc9': 0,
        'doc10': 0,
    },
    # ...
}

documents = [
    {'id': 'doc1', 'contents': 'Paris is the capital of France.'},
    {'id': 'doc2', 'contents': 'France is a country in Europe.'},
    {'id': 'doc3', 'contents': 'The Eiffel Tower is located in Paris.'},
    {'id': 'doc4', 'contents': 'Germany is a country in Europe.'},
    {'id': 'doc5', 'contents': 'The Louvre is a museum located in Paris.'},
    {'id': 'doc6', 'contents': 'The Catcher in the Rye is a novel by J.D. Salinger.'},
    {'id': 'doc7', 'contents': 'J.D. Salinger was an American writer.'},
    {'id': 'doc8', 'contents': 'The Catcher in the Rye is a coming-of-age novel.'},
    {'id': 'doc9', 'contents': 'To Kill a Mockingbird is a novel by Harper Lee.'},
    {'id': 'doc10', 'contents': 'Harper Lee was an American novelist.'},
    # ...
]

with open('json/output.json', 'w') as f:
    for doc in documents:
        f.write(json.dumps(doc) + '\n')




In [12]:
index_path = './indeks/'

!python -m pyserini.index.lucene \
    --collection JsonCollection \
    --input json/ \
    --index ./indeks/ \
    --generator DefaultLuceneDocumentGenerator \
    --threads 1 \
    --storeContents

2023-05-10 09:50:41,667 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: json/
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-05-10 09:50:41,668 INFO  [main] index.IndexCollection (IndexCollection.

In [31]:
import collections
import json
from pyserini.search import LuceneSearcher
import pytrec_eval
index_path = './indeks/'
# Load the index using a LuceneSearcher object

searcher = LuceneSearcher(index_path)

#Set the number of results to retrieve per query
hits_per_query = 10
searcher.set_bm25(0.9, 0.4)

# Define the queries, relevance judgments, and information need descriptions
queries = {
    'q1': 'What is the capital of France?',
    'q2': 'Who is the author of The Catcher in the Rye?',
    # ...
}

qrels = {
    'q1': {
        'doc1': 1,
        'doc2': 0,
        'doc3': 1,
        'doc4': 0,
        'doc5': 1,
    },
    'q2': {
        'doc6': 0,
        'doc7': 1,
        'doc8': 1,
        'doc9': 0,
        'doc10': 0,
    },
    # ...
}

information_need_descriptions = {
    'q1': 'Find information about the capital city of France.',
    'q2': 'Find information about the author of The Catcher in the Rye.',
    # ...
}

# Define the evaluation measures to compute
measures = {'recip_rank', 'map', 'ndcg'}

# Compute the query results and convert them to the format required by Pytrec_eval
results = {}
for query_id, query_text in queries.items():
    hits = searcher.search(query_text, k=hits_per_query)
    query_results = {hit.docid: {'rank': i+1, 'score': hit.score} for i, hit in enumerate(hits)}
    results[query_id] = query_results
print(results)
pytrec_eval_qrels = (qrels)

# Convert the results dictionary into the format required by Pytrec_eval
pytrec_eval_results = {}
for query_id, docs in results.items():
    for rank, (doc_id, doc_info) in enumerate(docs.items()):
        score = doc_info['score']
        pytrec_eval_results.setdefault(query_id, {})[doc_id] = score


# Create a Pytrec_eval evaluator object and compute the evaluation scores
evaluator = pytrec_eval.RelevanceEvaluator(pytrec_eval_qrels, measures)
evaluation_scores = evaluator.evaluate(pytrec_eval_results)

# Print the evaluation scores for each query
for query_id, query_text in queries.items():
    print(f"Evaluation results for query '{query_text}':")
    print(json.dumps(evaluation_scores[query_id], indent=2))
    print()


{'q1': {'doc1': {'rank': 1, 'score': 1.9193999767303467}, 'doc2': {'rank': 2, 'score': 0.8185999989509583}}, 'q2': {'doc6': {'rank': 1, 'score': 1.4889999628067017}, 'doc8': {'rank': 2, 'score': 1.4889990091323853}}}
Evaluation results for query 'What is the capital of France?':
{
  "map": 0.3333333333333333,
  "recip_rank": 1.0,
  "ndcg": 0.46927872602275644
}

Evaluation results for query 'Who is the author of The Catcher in the Rye?':
{
  "map": 0.25,
  "recip_rank": 0.5,
  "ndcg": 0.38685280723454163
}



In [32]:
# Create a Pytrec_eval evaluator object and compute the evaluation scores

# Get the MRR score from the Pytrec_eval results
mrr = pytrec_eval_results['recip_rank']['all']['mean_reciprocal_rank']
pytrec_eval_results


KeyError: 'recip_rank'